In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

project_path = os.path.abspath(os.path.join('..'))

if project_path not in sys.path:
    sys.path.append(project_path)

In [3]:
from src.features.generate_features import *
from src.models.model_pipeline import *
from src.preprocess.preprocess_ATSF import process
## change path
# from model_pipeline import *

In [4]:
## read raw data
data = pd.read_csv('../data/kaggle/data_kaggle.csv')

## custom process function, for your data rewrite process function
data = process(data)

In [5]:
## configs paths

config_filename = '../src/configs/gen_features_config.json'
model_config_filename = '../src/configs/model_config.json'

In [6]:
features_json = read_json(config_filename)
data[features_json['date_col']] = pd.to_datetime(data[features_json['date_col']], dayfirst=True)

In [7]:
## create features

data_features = generate_lagged_features(data=data,
                         target_cols=features_json['target_cols'],
                         id_cols=features_json['id_cols'],
                         date_col=features_json['date_col'],
                         lags=features_json['lags'],
                         windows=features_json['windows'],
                         preagg_methods=features_json['preagg_methods'],
                         agg_methods=features_json['agg_methods'],
                         dynamic_filters=features_json['dynamic_filters'],
                         ewm_params=features_json['ewm_params']
                        )

IntProgress(value=0, max=1)

In [8]:
## choose available models

choose_model(model_config_filename)

Choose model:


interactive(children=(Dropdown(description='model', options=('LSTM', 'GRU', 'TFT', 'DeepAR', 'NBeats'), value=…

In [9]:
model_config = read_json(model_config_filename)
forecast = pipeline(data_features, model_config)

Preprocessing:


  0%|          | 0/2640 [00:00<?, ?it/s]

Scaling:


  0%|          | 0/11 [00:00<?, ?it/s]

Training:
Epoch 0
---------


...

Train loss: 0.27819206469199237


  0%|          | 0/340 [00:00<?, ?it/s]

Smape = 86.33790989856902



  0%|          | 0/340 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [11]:
## check results

forecast.sample(3)

,SKU_id,Store_id,Date,Demand
5692,2,97,2016-06-11,50.006516
1193,1,43,2016-06-09,234.724335
2323,1,83,2016-06-19,391.046326


In [10]:
## save anywhere
# res_test[model_config['target']].to_csv('ans.csv', index_label='id')